In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
  model_name="gpt-4",
  openai_api_key=openai_api_key,
  temperature=0.1
)

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [3]:
from langchain.storage import LocalFileStore
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

In [23]:
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [25]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

chain = (
    RunnableMap({
        "context": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    })
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [26]:
invoke_chain("Is Aaronson guilty?")

content='Yes, according to the context, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [27]:
invoke_chain("What message did he write on the table?")

content='He traced the message "2+2=5" in the dust on the table.'


In [28]:
invoke_chain("Who is Julia?")

content='The context provided does not give specific information about who Julia is. However, it is mentioned that the main character had a strong emotional connection with her, experiencing a hallucination of her presence and expressing concern for her well-being. He also mentions her in a moment of fear and desperation.'
